In [22]:
import os
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base

In [23]:
lins = []
e = 0
with open('DB/00-common_all.vcf', 'r') as handle:
    for lin in handle:
        if e < 100:
            if not lin.startswith('#'):
                lins.append(lin)
                e+=1
        else:
            break


In [27]:
engine = create_engine('sqlite:///DB/dbsnp.db', echo=True)
Base = declarative_base()


class VariantInfo(Base):

    __tablename__ = "dbsnp"

    id = Column(Integer, primary_key=True)
    chromosome = Column(String)  
    coordinate = Column(Integer)
    rs = Column(String)
    refAllele = Column(String)
    altAllele = Column(String)
    refAlleleFrequency = Column(String)
    altAlleleFrequencySum = Column(String)


    def __init__(self, name):

        self.name = name    


Base.metadata.create_all(engine)

2024-07-11 16:45:32,065 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-11 16:45:32,066 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("dbsnp")
2024-07-11 16:45:32,066 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-11 16:45:32,067 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("dbsnp")
2024-07-11 16:45:32,068 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-11 16:45:32,069 INFO sqlalchemy.engine.Engine 
CREATE TABLE dbsnp (
	id INTEGER NOT NULL, 
	chromosome VARCHAR, 
	coordinate INTEGER, 
	rs VARCHAR, 
	"refAllele" VARCHAR, 
	"altAllele" VARCHAR, 
	"refAlleleFrequency" VARCHAR, 
	"altAlleleFrequencySum" VARCHAR, 
	PRIMARY KEY (id)
)


2024-07-11 16:45:32,069 INFO sqlalchemy.engine.Engine [no key 0.00033s] ()
2024-07-11 16:45:32,072 INFO sqlalchemy.engine.Engine COMMIT


/tmp/ipykernel_5000/369901646.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [34]:
e = 1
with sqlite3.connect('DB/dbsnp.db') as conn:
    for l in lins:
        ls = l.split('\t')
        info = ls[-1].split(';')
        for unit in info:
            if 'CAF' in unit:
                if len(unit.split(',')) > 2:
                    ref = float(unit.split('=')[1].split(',')[0])
                    maf = [a for a in unit.split('=')[1].split(',')[1:]]
                    totalMaf = 0
                    for i in maf:
                        if i == '.':
                            totalMaf+=0
                        else:
                            totalMaf+=float(i)

                else:
                    ref = float(unit.split('=')[1].split(',')[0])
                    alt = float(unit.split('=')[1].split(',')[1])
        row = [e, 'chr'+ls[0], ls[1], ls[2], ls[3], ls[4], ref, alt]
        e+=1
        add_rs(conn, row)

TypeError: add_rs() missing 1 required positional argument: 'task'

In [33]:
def add_rs(conn, task):
    sql = '''INSERT INTO dbsnp(id, chromosome, coordinate, rs, refAllele, altAllele, refAlleleFrequency, altAlleleFrequency)
             VALUES(?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid